In [ ]:
import h5py
import re
#import hickle as hkl
import torch
import torch.nn as nn
from torch.legacy.nn import Reshape
import graphviz
import torch.nn.functional as F
from torch.autograd import Variable
#from visualize import make_dot
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
import torch.utils.data as utils
import numpy as np
import matplotlib.pyplot as plt
from scipy.misc import imresize, imread, imshow
import time
import logging
from math import log,sqrt

In [ ]:
incep_v3 = models.inception_v3()


In [ ]:
print(incep_v3)


In [ ]:
# #incep_v3.Mixed_5b
# #list(incep_v3.AuxLogits.children())[:-1]
#############
# incep_v3.AuxLogits= nn.Sequential(*list(incep_v3.AuxLogits.children())[:-1])
#################
# # incep_v3= nn.Sequential(*list(incep_v3.children())[:-1])
# # incep_v3
# # incep_v3.training = True
# # incep_v3.aux_logits= True]
#vgg_16 = nn.Sequential(*list(vgg_16.children())[:-1])
#mod = nn.Sequential(*list(incep_v3.AuxLogits.children())[:-1])

In [ ]:
#incep_v3.AuxLogits
list(incep_v3.children())[14:17]

#list(incep_v3.children())[0:3]

In [ ]:
def InitializeWeights(mod):
    for m in mod.modules():
        if isinstance(m,nn.Conv2d):
            n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
            #print m.weight.size(), m.out_channels, m.in_channels
            m.weight.data.normal_(0,sqrt(2./n))
        elif isinstance(m, nn.BatchNorm2d):
            m.weight.data.fill_(1)
            m.bias.data.zero_()
        elif isinstance(m, nn.Linear):
            m.bias.data.zero_()
    return mod

In [ ]:
conv1_l = nn.Sequential(nn.BatchNorm2d(2048),nn.ReLU(),nn.Conv2d(2048,1024,1))
conv1_l = InitializeWeights(conv1_l)
conv2_l = nn.Sequential(nn.BatchNorm2d(1024),nn.ReLU(),nn.Conv2d(1024,512,(2,2),(1,2)))
conv2_l = InitializeWeights(conv2_l)
conv3_l = nn.Sequential(nn.BatchNorm2d(512),nn.ReLU(),nn.Conv2d(512,128,(3,3)))
conv3_l = InitializeWeights(conv3_l)
norm1_l = nn.BatchNorm2d(128)
norm1_l = InitializeWeights(norm1_l)
fc1_l = nn.Sequential(nn.Linear(256, 1))
fc1_l = InitializeWeights(fc1_l)
#############__________Aux_____________#############
conv1_aux = nn.Sequential(nn.BatchNorm2d(768),nn.ReLU(),nn.Conv2d(768,128,4,(1,2),(1,0)))
conv1_aux = InitializeWeights(conv1_aux)
conv2_aux = nn.Sequential(nn.BatchNorm2d(128),nn.ReLU(),nn.Conv2d(128,32,(1,2)))
conv2_aux = InitializeWeights(conv2_aux)
norm1_aux = nn.BatchNorm2d(32)
norm1_aux = InitializeWeights(norm1_aux)
fc1_aux = nn.Sequential(nn.Linear(640, 1))
fc1_aux = InitializeWeights(fc1_aux)


In [ ]:
class IV3_git(nn.Module):
    def __init__(self, pretrained_model):
        super(IV3_git, self).__init__()
        self.layer0_2 = nn.Sequential(*list(incep_v3.children())[0:3])#[0,1,2]
        self.layer3_4 =  nn.Sequential(*list(incep_v3.children())[3:5])#[3,4]
        self.layer5_12 =  nn.Sequential(*list(incep_v3.children())[5:13])#[5,12]
        self.layer14_16 =  nn.Sequential(*list(incep_v3.children())[14:17])#[14,6]
        ############
        self.aauxLogits = nn.Sequential(*list(incep_v3.AuxLogits.children())[:-1])
        #######################
        self.conv1_l = conv1_l
        self.conv2_l = conv2_l
        self.conv3_l = conv3_l
        self.norm1_l = norm1_l
        self.fc1_l = fc1_l
        ########################   
        self.conv1_aux = conv1_aux
        self.conv2_aux = conv2_aux
        #self.conv3_aux = conv3_aux
        self.norm1_aux = norm1_aux
        self.fc1_aux= fc1_aux
        
        
            
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features        
            
    def forward(self, x):
        y1 = self.layer0_2(x)
        y1 = F.max_pool2d(y1, kernel_size=3, stride=2)
        y1 = self.layer3_4(y1)
        y1 = F.max_pool2d(y1, kernel_size=3, stride=2)
        y1 = self.layer5_12(y1)
        #################
        #if self.aux_logits:
        aux = self.aauxLogits(y1)
        aux = self.conv1_aux(aux)
        aux = self.conv2_aux(aux)
        #aux = self.conv3_aux(aux)
        aux = self.norm1_aux(aux)
        aux = aux.view(-1, self.num_flat_features(aux))
        aux = self.fc1_aux(aux)
        ##################
        y1 = self.layer14_16(y1)
        #y1 = F.avg_pool2d(y1, kernel_size=8)
        y1 = F.dropout(y1)
        y1 = self.conv1_l(y1)
        y1 = self.conv2_l(y1)
        y1 = self.conv3_l(y1)
        y1 = self.norm1_l(y1)
        y1 = y1.view(-1, self.num_flat_features(y1))
        y1= self.fc1_l(y1)
        ##################
        
        return aux,y1


In [ ]:
incep_net = IV3_git(incep_v3)


In [ ]:
print(incep_net)

In [ ]:
input1=Variable(torch.Tensor(30,3,180,320))
tic=time.time()
aux,incep=incep_net(input1)
tac=time.time()
print("aux\t "+str(aux.size()))
print("incep\t "+str(incep.size()))
print(tac-tic)

In [ ]:
[1,2]

In [ ]:
sum1 = 0
        
print("Number of layers ---> ",len(list(net.parameters())))
for params in net.parameters():
    if params.requires_grad == True:
        sum1 += params.numel()
    
print("Total number of parameters ---> ",sum1)